# Single source shortest path

### Handle dependencies

In [1]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable


### Handle imports

In [2]:
import json
from pyspark.sql import SparkSession
from math import sqrt
from time import time

In [3]:
MAX_PARTITIONS = 12

spark = SparkSession.builder.config(
    'spark.default.parallelism',
    MAX_PARTITIONS
).getOrCreate()
context = spark.sparkContext

23/06/30 01:48:29 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.1.164 instead (on interface enp0s31f6)
23/06/30 01:48:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/30 01:48:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/30 01:48:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Load graph

In [4]:
GRAPH_PATH = 'graph_costs.json'


with open(GRAPH_PATH, 'r') as file:
    data = json.load(file)
    nodes = context.parallelize(data['nodes'])
    edges = context.parallelize(data['edges'])

### Define functions to be used

In [5]:
def init_costs(nodes, node):
    return nodes.map(
        lambda elem: (elem, 0) if elem == node else (elem, float('inf'))
    )


def sssp_step(costs, edges):
    new_costs = edges.map(
        lambda elem: (elem[0], (elem[1], elem[2]))
    ).join(
        costs
    ).map(
        lambda elem: (elem[1][0][0], elem[1][0][1] + elem[1][1])
    ).union(
        costs
    ).reduceByKey(
        lambda x, y: min(x, y)
    )
    return new_costs


def sssp_done(costs, new_costs):
    done = costs.join(
        new_costs
    ).map(
        lambda elem: elem[1][0] == elem[1][1]
    ).reduce(
        lambda x, y: x and y
    )
    return done


def run_sssp(nodes, edges, init_node):
    costs = init_costs(nodes, init_node)
    sssp_start = time()
    iteration = 1
    while True:
        start = time()
        new_costs = sssp_step(costs, edges)
        if sssp_done(costs, new_costs):
            break
        costs = new_costs
        print(f'Iteration {iteration} finished in {time() - start:.3f} sec.')
        iteration += 1
    print(f'SSSP finished in {time() - sssp_start:.3f}')
    return new_costs.collect()

In [6]:
costs = run_sssp(nodes, edges, 2)

Iteration 1 finished in 3.615 sec.
Iteration 2 finished in 1.736 sec.


Iteration 3 finished in 1.639 sec.


Iteration 4 finished in 1.592 sec.


Iteration 5 finished in 1.587 sec.


Iteration 6 finished in 1.674 sec.
Iteration 7 finished in 1.549 sec.


Iteration 8 finished in 1.548 sec.
Iteration 9 finished in 1.570 sec.
SSSP finished in 18.046


In [7]:
print(f"{'node':<10} | {'cost':<10}")
print(f"{'-' * 11}+{'-' * 11}")
for node, cost in costs:
    print(f'{node:<10} | {round(cost, 8):<10}')

node       | cost      
-----------+-----------
156        | 13        
408        | 12        
504        | 11        
360        | 10        
456        | 11        
324        | 10        
888        | 16        
684        | 7         
228        | 6         
36         | 13        
960        | 13        
972        | 14        
936        | 11        
576        | 13        
444        | 11        
240        | 11        
468        | 12        
708        | 10        
132        | 8         
432        | 9         
288        | 8         
720        | 13        
12         | 11        
24         | 11        
384        | 10        
780        | 9         
252        | 9         
312        | 6         
948        | 9         
732        | 12        
264        | 11        
876        | 11        
768        | 9         
144        | 6         
900        | 10        
552        | 13        
852        | 4         
648        | 11        
276        | 11        
516        | 9  